In [3]:
import pandas as pd
import pickle
import glob

In [13]:
routes = []
#combine all the pickles into one python list then pd.DataFrame
for pFile in glob.glob('routes/*.p'):
    [routes.append(i) for i in pickle.load( open( pFile, "rb" ) )]
    
routes = pd.DataFrame(routes)

In [18]:
timeBucketIds = {
    1528286400 : 1, # 8am Weekdays (6/6) - (morning rush-hours 6 - 10am)
    1528308000 : 2, # 2pm Weekdays (6/6) - (non-rush hours)
    1528322400 : 3, # 6pm Weekdays (6/6) - (night rush-hours 4 - 8pm)
    1528567200 : 0  # 2pm Weekends (6/9) - (non-rush hours)
} 

#append time bucket ID to congestionDistanceMatrix
routes['timeBucketId'] = routes.apply(lambda row: timeBucketIds[row['time']], axis = 1)
routes.tail()

,error,index,overview,route,time,timeBucketId
15594,False,1195,ipnwFzdwbMb@JVNJKh@uBXaAJ]|@Rj@HdARjB`@vABn@Fz...,"[{'distance': 21, 'duration': 8, 'polyline': '...",1528567200,0
15595,False,1196,ipnwFzdwbMb@JVNJKh@uBXaAJ]|@Rj@HdARjB`@j@CRBVB...,"[{'distance': 21, 'duration': 8, 'polyline': '...",1528567200,0
15596,False,1197,ipnwFzdwbMb@JVNJKh@uBXaAJ]|@Rj@HdARxEdA`LxCZN|...,"[{'distance': 21, 'duration': 8, 'polyline': '...",1528567200,0
15597,False,1198,ipnwFzdwbMb@JVNJKh@uBXaAJ]|@Rj@HdARjB`@vABn@Fz...,"[{'distance': 21, 'duration': 8, 'polyline': '...",1528567200,0
15598,False,1199,ipnwFzdwbMb@JVNJKh@uBXaAJ]|@Rj@HdARxEdAjIvBtA`...,"[{'distance': 21, 'duration': 8, 'polyline': '...",1528567200,0


In [28]:
congestionDistanceMatrix = pd.read_csv('congestionDistanceMatrix.csv')
#join routes with congestionDistanceMatrix to get from and to zones
routesMerge = routes.merge(congestionDistanceMatrix, left_on='index', right_on='index1')[['error','overview','index','route','timeBucketId','fromID','toID']]

In [30]:
routesMerge.tail()

,error,overview,index,route,timeBucketId,fromID,toID
15594,False,ipnwFzdwbMb@JVNJKh@uBXaAJ]|@Rj@HdARjB`@vABn@Fz...,1198,"[{'distance': 21, 'duration': 8, 'polyline': '...",0,13,219
15595,False,ipnwFzdwbMb@JVNJKh@uBXaAZsA[KeCYaC[_B[cDy@w@W}...,1199,"[{'distance': 21, 'duration': 8, 'polyline': '...",1,13,220
15596,False,ipnwFzdwbMb@JVNJKh@uBXaAJ]|@Rj@HdARxEdAjIvBtA`...,1199,"[{'distance': 21, 'duration': 8, 'polyline': '...",2,13,220
15597,False,ipnwFzdwbMb@JVNJKh@uBXaAZsA[KeCYaC[_B[cDy@w@W}...,1199,"[{'distance': 21, 'duration': 8, 'polyline': '...",3,13,220
15598,False,ipnwFzdwbMb@JVNJKh@uBXaAJ]|@Rj@HdARxEdAjIvBtA`...,1199,"[{'distance': 21, 'duration': 8, 'polyline': '...",0,13,220


In [38]:
# errors
routesMerge[routesMerge['error'] == 'ZERO_RESULTS']

,error,overview,index,route,timeBucketId,fromID,toID
5752,ZERO_RESULTS,,2638,{},1,43,103
5753,ZERO_RESULTS,,2638,{},2,43,103
5754,ZERO_RESULTS,,2638,{},3,43,103
5755,ZERO_RESULTS,,2638,{},0,43,103
6804,ZERO_RESULTS,,2901,{},1,45,103
6805,ZERO_RESULTS,,2901,{},2,45,103
6806,ZERO_RESULTS,,2901,{},3,45,103
6807,ZERO_RESULTS,,2901,{},0,45,103
8016,ZERO_RESULTS,,3204,{},1,48,103
8017,ZERO_RESULTS,,3204,{},2,48,103


In [67]:
#https://github.com/mgd722/decode-google-maps-polyline
# This function is free of any dependencies.
def decode_polyline_towkt(polyline_str):
    '''Pass a Google Maps encoded polyline string; returns list of lat/lon pairs'''
    index, lat, lng = 0, 0, 0
    coordinates = []
    changes = {'latitude': 0, 'longitude': 0}

    # Coordinates have variable length when encoded, so just keep
    # track of whether we've hit the end of the string. In each
    # while loop iteration, a single coordinate is decoded.
    while index < len(polyline_str):
        # Gather lat/lon changes, store them in a dictionary to apply them later
        for unit in ['latitude', 'longitude']: 
            shift, result = 0, 0

            while True:
                byte = ord(polyline_str[index]) - 63
                index+=1
                result |= (byte & 0x1f) << shift
                shift += 5
                if not byte >= 0x20:
                    break

            if (result & 1):
                changes[unit] = ~(result >> 1)
            else:
                changes[unit] = (result >> 1)

        lat += changes['latitude']
        lng += changes['longitude']

        coordinates.append((str(lng / 100000.0), str(lat / 100000.0)))
        
    #'LINESTRING(-71.160281 42.258729,-71.160837 42.259113,-71.161144 42.25932)'
    linestring = ''
    for pair in coordinates:
        linestring += ' '.join(pair) + ','
    return f"LINESTRING({linestring[:-1]})"

In [72]:
#flatten to leg/steps and decode polyline using function above
legs = []

def flattenAndAppend(row):
    for leg in row['route']:
        legs.append({
            'timeBucketId' : row['timeBucketId'],
            'fromID': row['fromID'],
            'toID': row['toID'],
            'distance': leg['distance'],
            'duration' : leg['duration'],
            'instructions': leg['instructions'],
            'polyline' : decode_polyline_towkt(leg['polyline'])
        })

routesMerge.apply(flattenAndAppend, axis = 1)

legs = pd.DataFrame(legs)

In [73]:
legs.tail()

,distance,duration,fromID,instructions,polyline,timeBucketId,toID
243367,15,17,13,Turn <b>left</b> onto <b>Broadway</b>,"LINESTRING(-73.90612 40.87708,-73.90618 40.877...",0,220
243368,561,126,13,Turn <b>right</b> onto <b>W 230th St</b>,"LINESTRING(-73.90622 40.87697,-73.90648 40.876...",0,220
243369,40,12,13,Turn <b>left</b> to stay on <b>W 230th St</b>,"LINESTRING(-73.91037 40.88036,-73.91037 40.880...",0,220
243370,189,30,13,Turn <b>left</b> onto <b>Irwin Ave</b>,"LINESTRING(-73.91074 40.88051,-73.91082 40.880...",0,220
243371,367,55,13,Turn <b>right</b> onto <b>Johnson Ave</b>,"LINESTRING(-73.91237 40.87935,-73.91241 40.879...",0,220


In [74]:
legs.at[1, 'polyline']

'LINESTRING(-74.01572 40.71171,-74.0158 40.71159,-74.01574 40.71153,-74.01534 40.71139,-74.01515 40.71132,-74.01491 40.71122,-74.01482 40.71119,-74.01467 40.71113)'

In [77]:
#export 
legs.to_csv('data/legs.csv', index = False)